# Load ImageNet
there are train and val subsets, each return a list of tuple with image path and label in integer

In [3]:
import os
import os.path as osp
import io
import PIL
from PIL import Image
import lmdb
from tqdm import tqdm
print(PIL.__version__)
print(lmdb.__version__)

7.0.0.post3
0.94


In [4]:
dataset_dir = "/media/allen/mass/DB/imagenet"
train_dir = osp.join(dataset_dir, "ilsvrc2012_train")
val_dir = osp.join(dataset_dir, "ilsvrc2012_val")
train_list = osp.join(dataset_dir, "ilsvrc2012_train.txt")
val_list = osp.join(dataset_dir, "ilsvrc2012_val.txt")
train_lmdb = osp.join(dataset_dir, "imagenet_lmdb_train")
val_lmdb = osp.join(dataset_dir, "imagenet_lmdb_val")

# Make LMDB
1. use PIL to load image and turn to byte string
2. tqdm to supervise the progress
3. use caffe datum as the serialized string to make value of LMDB

### making lmdb for train

In [13]:
lmdb_env = lmdb.open(train_lmdb_file, map_size=int(1e12))
lmdb_txn = lmdb_env.begin(write=True)

for img_path, label in tqdm(dataset.train):
    img = Image.open(img_path)
    img = img.convert("RGB")
    img = img.resize((w, h), Image.ANTIALIAS)
    datum.data = img.tobytes()
    datum.label = label
    key = pattern.search(img_path).group()
    lmdb_txn.put(key.encode(), datum.SerializeToString())
lmdb_txn.commit()

 78%|███████▊  | 1002031/1281167 [6:51:31<1:04:12, 72.45it/s]  /home/allen.jc.lin/.pyenv/versions/3.6.8/envs/py368/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:785: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))
100%|██████████| 1281167/1281167 [8:24:26<00:00, 42.33it/s]   


### making lmdb for val

In [5]:
lmdb_env = lmdb.open(val_lmdb, map_size=int(1e12))
lmdb_txn = lmdb_env.begin(write=True)
with open(val_list, 'r') as f:
    for line in tqdm(f.readlines()):
        img_path, label = line.strip().split(" ")
        img_full_path = osp.join(val_dir, img_path)
        f_img = open(img_full_path, 'rb')
        img_str = f_img.read()
        lmdb_txn.put(img_path.encode(), img_str)        
#         break
lmdb_txn.commit()

  0%|          | 0/50000 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: '/media/allen/mass/DB/imagenet/ilsvrc2012_val/ILSVRC2012_val_00000001.JPEG'

# Test the lmdb data

In [21]:
test_lmdb_env = lmdb.open(val_lmdb)
test_lmdb_txn = test_lmdb_env.begin()

with open(val_list, 'r') as f:
    for line in f.readlines():
        img_path, label = line.strip().split(" ")
        raw = test_lmdb_txn.get(img_path.encode())
        img_byte = io.BytesIO(raw)
        test_img = Image.open(img_byte)
        break

In [6]:
import random

In [9]:
random.choice([0,3])

3